# Data Science Couterfactual Problem Assignment

In [1]:
import os
import sys
import polars as pl
from typing import List, Dict, Callable, Any
from dataclasses import dataclass
from main import DataDictionary, Dataset, model
from variables import default_classifications
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# 0. Description and classification of data objects

In [2]:
# Creates data dictionary
dictionary:DataDictionary = DataDictionary()
# Assigns default classifications
dictionary.apply_assign(default_classifications,dictionary.assign_classification)
# Prints the entries as a dataframe
dictionary_df:pl.DataFrame = dictionary.frame_dictionary()

# 1. Model the right target

In [ ]:
dataset = Dataset()
data_preview:pl.DataFrame = dataset.preview()
data_table_stats:pl.DataFrame = (
    dataset.data.describe()
)

# 2. Train a probabilistic model

You want P(success | features), not just a hard label.

- Logistic regression (baseline, interpretable)

- Gradient Boosted Trees (XGBoost / LightGBM) ← best tradeoff

- Random Forest (okay, slower, less sharp)

Scikit-learn is the library to use.

## Logistic Regression

In [4]:
# logistic_model = model()
# logistic_regression = logistic_model.gb_trees()

## Gradient Boosted Trees

In [ ]:
xgb_model = model()

xboosted_model = xgb_model.gb_trees(
    cat_cols=[], # maybe making a categorical column will help...
    num_cols = [col for col, dtype in dataset.data.schema.items() if dtype.is_numeric() and col not in ["active_account", "delinquent_account"]],
)
# 0.5 is random chance
# Closer to 1 is better

ROC AUC: 0.5
              precision    recall  f1-score   support

           0       0.50      0.33      0.40         3
           1       0.71      0.83      0.77         6

    accuracy                           0.67         9
   macro avg       0.61      0.58      0.58         9
weighted avg       0.64      0.67      0.65         9



## Random Forest

# 3. Define Constraints Explicitly

Defining how to change the logistic regression. (Credit score can't be 1)

In [ ]:
constraints = {
    "ident_monitor_opt": [0, 1],
    "income": [4000,136000],
    "days_from_registration": [42,4592],
    "days_from_login": [1,424],
    "max_bkaccts": [1,4],
}
change_cost = {
    "ident_monitor_opt": 1,
    "income": 1,
    "days_from_registration": 1,
    "days_from_login": 1,
    "max_bkaccts": 1,
}


# 4. Constrained Search

Trying a bunch of different combinations looking for the optimal combination.

In [ ]:
import itertools
import numpy as np

def best_counterfactual(row, model, budget=2):
    best = None
    best_p = -1

    for combo in itertools.product([0,1], repeat=len(bool_features)):
        candidate = row.copy()

        cost = 0
        for f, v in zip(bool_features, combo):
            if candidate[f] != v:
                cost += change_cost[f]
                candidate[f] = v

        if cost > budget:
            continue

        p = model.predict_proba(candidate.values.reshape(1, -1))[0,1]
        if p > best_p:
            best_p = p
            best = candidate

    return best, best_p


# 5. Smarter Search

# 6. Explanation

# 7. Possible Problems

# 8. Future Features (Optional)